# Importing MI-FNO fragments from QEMIST Cloud

(Copy/paste from abstract)
The method of increment (MI) approach expresses the electron correlation energy of a molecular system as a truncated many-body expansion in terms of orbitals, atoms, molecules, or fragments. The electron correlation of the system is expanded in terms of occupied orbitals, and the MI approach is employed to systematically reduce the occupied orbital space. At the same time, the virtual orbital space is reduced based on the frozen natural orbitals (FNO), which are obtained using a one-particle density matrix from second-order, many-body perturbation theory. In this way, a method referred to as the MI-FNO approach is available for the systematic reduction of both the occupied space and the virtual space in quantum chemistry simulations. 

<div>
<img src="img/mifno.png" width="700"/>
</div>

The MI approach, first introduced in quantum chemistry by Nesbet ([Phys. Rev. 1967, 155, 51](https://doi.org/10.1103/PhysRev.155.51), [Phys. Rev. 1967, 155, 56](https://doi.org/10.1103/PhysRev.155.56) and [Phys. Rev. 1968, 175, 2](https://doi.org/10.1103/PhysRev.175.2)), is based upon the n-body Bethe–Goldstone expansion ([Proc. R. Soc. A, 1957, 238, 551](https://doi.org/10.1098/rspa.1957.0017)) of the correlation energy of a molecule. The correlation energy ($E_c$), defined as the difference between the exact ($E_{\text{exact}}$) and the Hartree–Fock (mean-field) energy ($E_{\text{HF}}$), can be expanded as
$$
\begin{align*}
E_c &= E_{\text{exact}} - E_{\text{HF}} \\
&= \sum_i \epsilon_i + \sum_{i>j} \epsilon_{ij} + \sum_{i>j>k} \epsilon_{ijk} + \sum_{i>j>k>l} \epsilon_{ijkl} + \dots
\end{align*}
$$

where $\epsilon_i$, $\epsilon_{ij}$, $\epsilon_{ijk}$, and $\epsilon_{ijkl}$ are, respectively, the one-, two-, three-, and four-body increments (expansions) defined as
$$
\begin{align*}
\epsilon_i &= E_c(i) \\
\epsilon_{ij} &= E_c(ij) - \epsilon_i - \epsilon_j \\
\epsilon_{ijk} &= E_c(ijk) - \epsilon_{ij} - \epsilon_{ik} - \epsilon_{jk} - \epsilon_{i} - \epsilon_{j} - \epsilon_{k} \\
\epsilon_{ijkl} &= E_c(ijkl) - \epsilon_{ijk} - \epsilon_{ijl} - \epsilon_{jkl} - \dots \\
&\vdots
\end{align*}
$$

The subproblems resulting from the MI-FNO reduction can then be solved by any algorithm, including quantum algorithms such as the phase estimation algorithm and the variational quantum eigensolver, to predict the correlation energies of a molecular system.

QEMIST Cloud a pipeline 

## System definition

- Definition of the molecular system
- HF 10é system

In [ ]:
HF_coordinates = """
    F    0.0000   0.0000   0.0000
    H    0.0000   0.0000   0.9168
"""
basis = "cc-pvdz"
charge = 0 
spin = 0

## MI-FNO calculation

- Installation of qemist + tangelo
- Setting up env variables
- Doc of qemist https://alpha-api.qemist.cloud/#/docs (I am writing a page)
- Not executed now, retrieving a json dictionary.

```python
import os
os.environ['QEMIST_PROJECT_ID'] = "your_project_id_string"
os.environ['QEMIST_AUTH_TOKEN'] = "your_qemist_authentication_token"

import qemist_client
from qemist.problem_decomposition import IncrementalDecomposition
from qemist.problem_reduction import FNO
from qemist.electronic_structure_solvers import HBCI

# Create the QEMIST molecule object.
HF_mol = qemist_client.molecule.Molecule(atom=HF_coordinates)
HF_mol.basis = basis
HF_mol.charge = charge
HF_mol.spin = spin

# Defining the solvers. For more information, refer to the QEMIST Cloud documentation.
hbci_solver = HBCI()
fno = FNO(hbci_solver, export_fragment_data=True)
mi_solver = IncrementalDecomposition(electronic_structure_solver=fno,
                                     truncation_order=2)
# Submit the problem to the cloud.
HF_handle = mi_solver.simulate(system=HF_mol)
print(f"\nHF handle: {HF_handle}")

# Retrieving the results.
HF_result = mi_solver.get_result()
```

The `HF_result` can be saved in a json file with the following code snipplet.
```python
import json
with open("your_file_path", "w") as f:
    json.dump(HF_result, f)
```

## Import MI-FNO fragments into Tangelo

In [ ]:
from tangelo.problem_decomposition import MIFNOHelper

In [ ]:
fno_fragments = MIFNOHelper(json_file="./data/HF_MIFNO_trunc2.json")
print(fno_fragments)

### Downloading molecular coefficients files

In [ ]:
# Creating of a mo_coeff folder.
os.mkdir("./mo_coeff")

# Downloading the HDF5 files to the 
fno_fragments.retrieve_mo_coeff("./mo_coeff")

### Reconstructing a fragment Hamiltonian

In [ ]:
from tangelo import SecondQuantizedMolecule
from tangelo.toolboxes.qubit_mappings.mapping_transform import fermion_to_qubit_mapping

mol = SecondQuantizedMolecule(HF_coordinates, q=charge, spin=spin, basis=basis)

ferm_op = fno_fragments.compute_fermionoperator(mol, "(1, 2)")
qu_op = fermion_to_qubit_mapping(ferm_op, mapping="JW")

In [ ]:
#e = fno_fragments.mi_summation()#{"(1, 2)": -15.56})
#print(e)

## VQE on a Fragment

In [ ]:
from openfermion.linalg import qubit_operator_sparse